# 

In [21]:
# --- Fase 1: Usando a Biblioteca GoogleNews (com Paginação) ---
try:
    from GoogleNews import GoogleNews
except ImportError:
    try:
      from pygooglenews import GoogleNews
    except ImportError:
      print("Biblioteca GoogleNews ou pygooglenews não encontrada.")
      print("Instale com: pip3 install GoogleNews OU pip3 install pygooglenews")
      news_df = pd.DataFrame()

import pandas as pd
import time # Importar a biblioteca time para pausas

all_results = [] # Lista para guardar resultados de todas as páginas

try:
    # --- ALTERAÇÃO AQUI: Período pode ser menor agora, ex: '7d' ou '30d' ---
    googlenews = GoogleNews(lang='en', period='90d') # Mantemos 30d por enquanto
    query = 'NVIDIA Stock'

    # --- Loop de Paginação ---
    print(f"Buscando notícias para '{query}':")
    for page in range(1, 10): # Tenta buscar da página 1 até a 5
        print(f"Buscando página {page}...")
        googlenews.search(query)
        googlenews.get_page(page) # Tenta ir para a página específica
        
        page_results = googlenews.result()
        if not page_results: # Se não houver resultados na página, para o loop
            print(f"Página {page} não retornou resultados. Interrompendo.")
            break
            
        all_results.extend(page_results) # Adiciona os resultados da página à lista geral
        time.sleep(0.5) # Pequena pausa para não sobrecarregar o Google

    # --- Processamento dos Resultados Combinados ---
    if all_results:
        news_df = pd.DataFrame(all_results)
        
        # Remove duplicatas baseadas no título
        news_df = news_df.drop_duplicates(subset=['title'])

        if 'title' in news_df.columns:
            news_df = news_df[['title']].rename(columns={'title': 'Title'})
            print(f"\nEncontradas {len(news_df)} manchetes únicas após buscar {page-1 if page > 1 else 1} página(s).")
            display(news_df.head())
        else:
            print("Não foram encontradas manchetes ou a coluna 'title'.")
            news_df = pd.DataFrame()
    else:
        print("Nenhuma manchete encontrada após a busca paginada.")
        news_df = pd.DataFrame()

except NameError:
    pass
except AttributeError:
    print("\nERRO: A biblioteca instalada pode não ter o método 'get_page()'.")
    print("Verifique a documentação da versão que você instalou.")
    news_df = pd.DataFrame() # Garante que news_df exista
except Exception as e:
    print(f"\nOcorreu um erro durante a busca paginada: {e}")
    news_df = pd.DataFrame()

Buscando notícias para 'NVIDIA Stock':
Buscando página 1...
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
Página 1 não retornou resultados. Interrompendo.
Nenhuma manchete encontrada após a busca paginada.


In [20]:
# --- Fase 2: Análise de Sentimento com VADER ---

# Importando a ferramenta de análise de sentimento
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd # Reimportar pandas caso necessário

analyzer = SentimentIntensityAnalyzer()


def calculate_sentiment(text):
    vs = analyzer.polarity_scores(text)
    return vs['compound']

if not news_df.empty:
    news_df['Sentiment'] = news_df['Title'].apply(calculate_sentiment)

    print("\nDataFrame com as pontuações de sentimento:")
    display(news_df)
else:
    print("\nDataFrame de notícias está vazio. Não foi possível calcular o sentimento.")


DataFrame com as pontuações de sentimento:


,Title,Sentiment
0,Nvidia leads $260 mn round in IIT-Madras incub...,0.0000
1,AI Heavyweight Nvidia Leads 15 Hot Stocks Onto...,0.6369
2,"Líderes em IA e dados NVIDIA, AMD, Snowflake e...",0.0000
3,NVIDIA Corporation (NVDA): A Bull Case Theory,0.0000
4,"Like it or not, we’re all living in the AI eco...",0.3612
...,...,...
895,The 3 largest individual stock holdings in my ...,0.0000
896,Is There Still an Opportunity for Investors Af...,0.4215
897,Intel’s Big Investors Set The Stage For A Make...,0.0000
898,2 Brilliant Growth Stocks to Buy Now and Hold ...,0.7506


In [19]:
# --- Fase 3: Coleta dos Dados de Mercado (NVDA) - CORRIGIDO ---

import yfinance as yf
import pandas as pd

# --- Passagem 1: Baixando os dados da NVDA ---
ticker_stock = 'NVDA'
end_date = pd.Timestamp.today()
start_date = end_date - pd.Timedelta(days=100)

print(f"Buscando dados da {ticker_stock} de {start_date.date()} até {end_date.date()}...")
# yfinance já ajusta os preços por padrão (auto_adjust=True)
nvda_df = yf.download(ticker_stock, start=start_date, end=end_date)

# --- Passagem 2: Calculando os Retornos Diários (usando 'Close') ---
# Como 'Close' já está ajustado, usamos ele para calcular o retorno
nvda_df['Daily_Return'] = nvda_df['Close'].pct_change() # <-- CORREÇÃO AQUI

# --- Passagem 3: Preparando o DataFrame de Retornos ---
nvda_returns = nvda_df[['Daily_Return']].dropna()
nvda_returns.index.name = 'Date'

print("\nDataFrame com os retornos diários da NVDA:")
display(nvda_returns.head())

Buscando dados da NVDA de 2025-07-14 até 2025-10-22...


/var/folders/bc/3fck38_57j918mwlyxrxlxnh0000gn/T/ipykernel_10976/2842623177.py:13: FutureWarning: YF.download() has changed argument auto_adjust default to True
  nvda_df = yf.download(ticker_stock, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed


DataFrame com os retornos diários da NVDA:


Price,Daily_Return
Ticker,
Date,
2025-07-16,0.003925
2025-07-17,0.009512
2025-07-18,-0.003410
2025-07-21,-0.005974
2025-07-22,-0.025382


In [18]:
#Fase 4: Agregando Sentimento e Juntando Dados
try:
    try:
        from GoogleNews import GoogleNews
    except ImportError:
        from pygooglenews import GoogleNews
    import time
    from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

    googlenews = GoogleNews(lang='en', period='90d')
    query = 'NVIDIA Stock'
    all_results = []
    print("\nRebuscando notícias para obter datas...")
    for page in range(1, 10):
        googlenews.search(query)
        googlenews.get_page(page)
        page_results = googlenews.result(sort=True)
        if not page_results: break
        all_results.extend(page_results)
        time.sleep(1)

    if all_results:
        news_df_full = pd.DataFrame(all_results)
        news_df_full = news_df_full.drop_duplicates(subset=['title'])
        print(f"Total de {len(news_df_full)} notícias com detalhes.")

        date_column_name = 'datetime' # <- VERIFIQUE ESTE NOME!

        if date_column_name not in news_df_full.columns:
            print(f"\nERRO CRÍTICO: Coluna de data '{date_column_name}' não encontrada!")
            print("Verifique as colunas disponíveis:", news_df_full.columns)
            raise KeyError(f"Coluna '{date_column_name}' não encontrada.")

        news_df_full['Date'] = pd.to_datetime(news_df_full[date_column_name]).dt.date
        news_df_full['Date'] = pd.to_datetime(news_df_full['Date'])

        analyzer = SentimentIntensityAnalyzer()
        news_df_full['Sentiment'] = news_df_full['title'].apply(calculate_sentiment)

        # --- Passagem 2: Calcular o Sentimento Médio por Dia ---
        daily_sentiment = news_df_full.groupby('Date')['Sentiment'].mean().reset_index()
        print("\nSentimento médio por dia:")
        display(daily_sentiment.head())

        # Garante que nvda_returns tenha um índice simples e 'Date' como coluna
        if isinstance(nvda_returns.index, pd.MultiIndex):
             print("Índice de nvda_returns é MultiIndex, resetando...")
             nvda_returns_flat = nvda_returns.reset_index()
        elif nvda_returns.index.name == 'Date':
             nvda_returns_flat = nvda_returns.reset_index()
        else:
            # Se já for índice simples sem nome 'Date', apenas copia
            nvda_returns_flat = nvda_returns.reset_index() # Resetar de qualquer forma é seguro

        # Garante que daily_sentiment tenha um índice simples
        daily_sentiment_flat = daily_sentiment.reset_index(drop=True) # drop=True evita criar coluna 'index'

        # Verifica os nomes das colunas de data (devem ser 'Date' em ambos)
        print("\nColunas antes do merge:")
        print("Retornos:", nvda_returns_flat.columns)
        print("Sentimento:", daily_sentiment_flat.columns)

        # Garante que ambas as colunas 'Date' sejam do tipo datetime
        nvda_returns_flat['Date'] = pd.to_datetime(nvda_returns_flat['Date'])
        daily_sentiment_flat['Date'] = pd.to_datetime(daily_sentiment_flat['Date'])


        print("\nTentando fazer o merge...")
        merged_df = pd.merge(nvda_returns_flat, daily_sentiment_flat, on='Date', how='left')
        print("Merge bem-sucedido!")

        merged_df['Sentiment'] = merged_df['Sentiment'].fillna(0)

        print("\nDataFrame final combinado (Retornos e Sentimento):")
        display(merged_df.head())

    else:
        print("Nenhuma notícia encontrada para processar.")
        merged_df = pd.DataFrame()

except NameError as e:
     print(f"Erro: Bibliotecas não importadas ou variável não definida. Detalhe: {e}")
     merged_df = pd.DataFrame()
except KeyError as e:
     print(f"Erro CRÍTICO ao processar DataFrame: {e}. Verifique nomes de colunas.")
     merged_df = pd.DataFrame()
except Exception as e:
     print(f"Ocorreu um erro inesperado: {e}")
     merged_df = pd.DataFrame()


Rebuscando notícias para obter datas...
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
Nenhuma notícia encontrada para processar.
